In [189]:
import kidcalc
import numpy as np
import scipy.integrate as integrate
from scipy.integrate import odeint
from scipy.optimize import minimize_scalar as minisc
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

class KID(object):
    """A class to make KID objects"""
    def __init__(self,Qc=2*10**4,hw0 = .6582*2*np.pi*5.28,kbT=.2*86,
                 V=10.**3):
        self.Qc = Qc #arb.
        self.hw0 = hw0 #ueV 
        self.kbT = kbT #ueV
        self.V = V # um^3
        
        #All for Al:
        self.kbTc = 100.8 #ueV
        self.D0 = 1.76*self.kbTc #ueV
        self.kbTD = 37312. #ueV
        self.t0 = 440 #ns
        self.tesc = .17 #ns
        self.tpb = .11 #ns
        self.ak = .1 #arb.
        self.d = .06 #um
        self.lbd0 = .092 #um
        self.N0 = 1.74*10**4 #ueV^-1 um^-3

### Calculated attributes
    @property
    def Vsc(self):
        def integrand1(E,D):
            return 1/np.sqrt(E**2-D**2)
        return 1/(integrate.quad(integrand1,self.D0,
                              self.kbTD,args = (self.D0,))[0]*self.N0) 
    @property
    def D_0(self):
        return kidcalc.D(self.kbT,self.N0,self.Vsc,self.kbTD)
    
    @property #takes .5s to calculate
    def hwread(self):
        return kidcalc.hwread(self.hw0,self.ak,
                             self.lbd0,self.d,self.D_0,
                             self.D0,self.kbT)
    @property
    def Nqp_0(self):
        return self.V*kidcalc.nqp(self.kbT,self.D_0,self.N0)
    
    @property
    def tqp_0(self):
        return (self.V*self.t0*self.N0*self.kbTc**3/
                (2*self.D0**2*self.Nqp_0))
    @property
    def tres(self):
        hwread = self.hwread
        s_0 = kidcalc.cinduct(hwread,self.D_0,self.kbT)
        Qi_0 = kidcalc.Qi(s_0[0],s_0[1],self.ak,
                          self.lbd0,self.d,self.D_0,self.D0,
                          self.kbT)
        Q = self.Qc*Qi_0/(self.Qc+Qi_0)
        return 2*Q/(hwread/.62)
        
    
### Calculation functions
    def calc_Nqpevol(self,Nqp_ini,tStop = None,tInc = None):
        if tStop is None:
            tStop = 2*self.tqp_0
        if tInc is None:
            tInc = tStop/100
        
        #### Calculate N_qp evolution
        def rateeq(N,t,params):
            N_qp, N_w = N
            R, V, G_B, G_es, N_w0 = params
            derivs = [-R*N_qp**2/V + 2*G_B*N_w, 
                     R*N_qp**2/(2*V) - G_B*N_w - G_es*(N_w - N_w0)]
            return derivs

        # Parameters
        R = (.5*(2*self.D0/self.kbTc)**3/
             (2*self.D0*self.N0*self.t0)) # ns^-1*um^3 (From Wilson2004 or 2.29)
        G_B = 1/self.tpb # ns^-1 (From chap8)
        G_es = 1/self.tesc  # ns^-1 (From chap8)

        N_w0 = R*self.Nqp_0**2*self.tpb/(2*self.V) # arb. 

        params = [R, self.V, G_B, G_es, N_w0]

        # Initial values
        N_0 = [Nqp_ini, N_w0]

        # Time array
        t = np.arange(0., tStop, tInc)

        return t,odeint(rateeq, N_0, t, args=(params,))
    
    def calc_linNqpevol(self,Nqp_ini,tStop = None,tInc = None):
        if tStop is None:
            tStop = 2*self.tqp_0
        if tInc is None:
            tInc = tStop/200
        t = np.arange(0., tStop, tInc)
        
        tqp_1 = .5*self.tqp_0*(1+self.tesc/self.tpb)
        return (Nqp_ini-self.Nqp_0)*np.exp(-t/tqp_1)
    
    def calc_resNqpevol(self,t,Nqpt,hwread):
        tres = self.tres
        X = np.exp(-t/tres)/np.sum(np.exp(-t/tres))
        dNqpt = np.convolve(Nqpt-self.Nqp_0,X)[:len(t)]
        return dNqpt+self.Nqp_0
    
    def calc_S21(self,Nqp,hwread, dhw = 0):
        kbTeff = kidcalc.kbTeff(Nqp,self.N0,self.V,self.Vsc,
                                self.kbTD,self.kbTc)
        D = kidcalc.D(kbTeff,self.N0,self.Vsc,self.kbTD)
        s1,s2 = kidcalc.cinduct(hwread + dhw,D,kbTeff) 
        Qi = kidcalc.Qi(s1,s2,self.ak,self.lbd0,self.d,D,
                        self.D0,self.kbT)
        hwres = kidcalc.hwres(s2,self.hw0,self.ak,
                              self.lbd0,self.d,D,
                              self.D0,self.kbT)
        return kidcalc.S21(Qi,self.Qc,hwread,dhw,hwres)
    
    def calc_resp(self,Nqp,hwread,dhw = 0):
        S21 = self.calc_S21(Nqp,hwread,dhw)
        s_0 = kidcalc.cinduct(hwread,self.D_0,self.kbT)
        Qi_0 = kidcalc.Qi(s_0[0],s_0[1],self.ak,
                          self.lbd0,self.d,self.D_0,self.D0,
                          self.kbT)
        
        S21min = (self.Qc*Qi_0)/(self.Qc+Qi_0)/Qi_0
        xc = (1+S21min)/2
        
        dA = 1 - np.sqrt((np.real(S21)-xc)**2+
                         np.imag(S21)**2)/(1-xc)
        theta = np.arctan2(np.imag(S21),(xc-np.real(S21)))
        return S21,dA,theta
    
    def calc_linresp(self,Nqp,hwread):
        kbTeff = kidcalc.kbTeff(Nqp,self.N0,self.V,self.Vsc,
                                self.kbTD,self.kbTc)
        D = kidcalc.D(kbTeff,self.N0,self.Vsc,self.kbTD)
        s1,s2 = kidcalc.cinduct(hwread,D,kbTeff)
        s_0 = kidcalc.cinduct(hwread,self.D_0,self.kbT)
        Qi_0 = kidcalc.Qi(s_0[0],s_0[1],self.ak,
                          self.lbd0,self.d,self.D_0,self.D0,
                          self.kbT)
        Q = Qi_0*self.Qc/(Qi_0+self.Qc)
        beta = 2*self.d/(self.lbd0*np.sinh(2*self.d/self.lbd0))
        lindA = -1*-self.ak*beta*Q*(s1 - s_0[0])/s2
        lintheta = -self.ak*beta*Q*(s2 - s_0[1])/s2
        return lindA, lintheta
    
      
### Plot functions    
    def plot_freqsweep(self,start=None,stop=None,points = 50):
        hwread = self.hwread
        s_0 = kidcalc.cinduct(hwread,self.D_0,self.kbT)
        Qi_0 = kidcalc.Qi(s_0[0],s_0[1],self.ak,
                          self.lbd0,self.d,self.D_0,self.D0,
                          self.kbT)
        Q = Qi_0*self.Qc/(Qi_0+self.Qc)
        
        
        if start is None:
            start = -self.hw0/Q*2
        if stop is None:
            stop = self.hw0/Q*2
            
        
        for dhw in np.linspace(start,stop,points):
            S21_0 = self.calc_S21(self.Nqp_0,hwread,dhw)
            plt.plot(np.real(S21_0),np.imag(S21_0),'r.')
    
    def plot_S21resp(self,Nqp_ini,tStop = None,tInc = None,
                       points = 10):
        Nqpt = self.calc_Nqpevol(Nqp_ini,tStop,tInc)[:,0]
        Nqpts = Nqpt[np.round(np.linspace(0,len(Nqpt)-1,points)
                             ).astype(int)]
        hwread = self.hwread
        
        for Nqp in Nqpts:
            S21 = self.calc_S21(Nqp,hwread)
            plt.plot(np.real(S21),np.imag(S21),'.b')
    
    def plot_dAthetaresp(self,Nqp_ini,tStop = None,tInc = None,
                       points = 50):
        if tStop is None:
            tStop = self.tqp_0
        if tInc is None:
            tInc = tStop/100
        hwread = self.hwread
        
        t, Nqpt = self.calc_Nqpevol(Nqp_ini,tStop = 8*10**9,
                                    tInc = 10**8)[:,0]
        
        Nqpts = Nqpt[np.rint(np.linspace(0,len(Nqpt)-1,points)
                            ).astype(int)]
        ts = t[np.rint(np.linspace(0,len(Nqpt)-1,points)).astype(int)]
        
        i = 0
        for Nqp in Nqpts:
            S21,dA,theta = self.calc_resp(Nqp,hwread)
            plt.figure(1)
            plt.plot(ts[i],dAtheta[0],'r.')
            plt.figure(2)
            plt.plot(ts[i],dAtheta[1],'b.')
            i += 1
            
    def plot_Nqpt(self,Nqp_ini,tStop = None,tInc = None, 
                  plot_phonon = False, fit_secondhalf = False, 
                  plot_lin = True):
        
        if tStop is None:
            tStop = 2*self.tqp_0
        if tInc is None:
            tInc = tStop/100
            
        t, Nqpevol = self.calc_Nqpevol(Nqp_ini,tStop,tInc)
        Nqpt = Nqpevol[:,0]
        Nwt = Nqpevol[:,1]
        
        plt.plot(t,Nqpt-self.Nqp_0)
        plt.yscale('log')
        
        if plot_lin:
            Nqptlin = self.calc_linNqpevol(Nqp_ini,tStop,tInc)
            plt.plot(t,Nqptlin)
        
        
        if fit_secondhalf:
            fit = curve_fit(lambda x,a,b: b*np.exp(-x/a),
                    t[np.round(len(t)/2).astype(int):],
                    Nqpt[np.round(len(t)/2).astype(int):]-self.Nqp_0,
                   p0 = (self.tqp_0,Nqp_ini-self.Nqp_0))
            print(fit[0][0])
            plt.plot(t,fit[0][1]*np.exp(-t/fit[0][0]))
        
        if plot_phonon:
            plt.figure()
            plt.plot(t,Nwt)
            plt.yscale('log')
        
            
    def plot_resp(self,Nqp_ini,tStop = None,tInc = None,
                       points = 50):
        if tStop is None:
            tStop = 2*self.tqp_0
        if tInc is None:
            tInc = tStop/100
        hwread = self.hwread
        
        t,Nqpwt = self.calc_Nqpevol(Nqp_ini,tStop,tInc)
        
        resNqpt = self.calc_resNqpevol(t,Nqpwt[:,0],hwread)
           
        Nqpts = resNqpt[np.rint(np.linspace(0,len(Nqpt)-1,points)
                               ).astype(int)]
        ts = t[np.rint(np.linspace(0,len(Nqpt)-1,points)).astype(int)]
        
        plt.figure(1,figsize = (5,5))
        
        self.plot_freqsweep()
        i = 0
        for Nqp in Nqpts:
            S21,dA,theta = self.calc_resp(Nqp,hwread)
            lindA,lintheta = self.calc_linresp(Nqp,hwread)

            plt.figure(1)
            plt.plot(np.real(S21),np.imag(S21),'.b')
            plt.figure(2)
            plt.plot(ts[i],dA,'r.')
            plt.plot(ts[i],lindA,'b.')
            plt.figure(3)
            plt.plot(ts[i],theta,'r.')
            plt.plot(ts[i],lintheta,'b.')
            i += 1
            
        plt.figure(4)
        self.plot_Nqpt(Nqp_ini,tStop,tInc)
            
        #Cosmetics
        plt.figure(1)
        plt.xlabel('Re(S21)')
        plt.ylabel('Im(S21)')
        plt.figure(2)
        plt.xlabel('t (ns)')
        plt.ylabel(r'$\delta A$')
        plt.yscale('log')
        plt.figure(3)
        plt.xlabel('t (ns)')
        plt.ylabel(r'$\theta$')
        plt.yscale('log')
        plt.figure(4)
        plt.ylabel(r'$\Delta N_{qp}$')
        plt.xlabel('t (ns)')
        plt.yscale('log')    